I scrape the table from wiki page. 
check and found that the neighborhood is already combined by PostCode
check the 

In [1]:
import pandas as pd

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 

In [36]:
df = dfs[0]
df.columns = ['PostalCode','Borough','Neighborhood']
df.head(10)

,PostalCode,Borough,Neighborhood
0,Postal Code,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


In [37]:
df = df.drop(index=0) # delete one row
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [42]:
print(df[df.PostalCode=='M5A'])
print(df.PostalCode.count()==len(df))
# so it already satisfy

  PostalCode           Borough               Neighborhood
5        M5A  Downtown Toronto  Regent Park, Harbourfront
True


In [31]:
df_part = df[df.Neighborhood =='Not assigned']
df_part

df_part[df_part.Borough != 'Not assigned']
df_part
df_part = df_part[df_part.Borough != 'Not assigned']
df_part
#so there is no Borough!= Not assigned' and Neighborhood==Not assinged

,PostalCode,Borough,Neighborhood


In [46]:
df.shape

(180, 3)